In [1]:
from torch.nn import functional, Sequential, Conv2d, ReLU, AvgPool2d
from torchvision import transforms
import torch.optim as optim
from torch import nn
from torch import cuda
import torchvision
import torch


In [2]:
devices = torch.device('cuda' if cuda.is_available() else 'cpu')
print(devices)

cuda


In [3]:
size = (64,64)
trans = transforms.Compose(
    [
        transforms.Resize(size=size),
        transforms.CenterCrop(50),
        transforms.ColorJitter(brightness=.5,contrast=.4,hue=.6),
        transforms.RandomEqualize(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ]
)

In [4]:
trainPath = './dataSet/Training/'
testPath = './dataSet/Test/'
varPath = './dataSet/Validation/'

In [5]:
trainData = torchvision.datasets.ImageFolder(root=trainPath,transform=trans)
testData = torchvision.datasets.ImageFolder(root=testPath,transform=trans)
varData = torchvision.datasets.ImageFolder(root=varPath,transform=trans)

In [6]:
from torch.utils import data

batch_size = 60
train_data_loader = data.DataLoader(trainData,batch_size=batch_size)
val_data_loader = data.DataLoader(varData,batch_size=batch_size)
test_data_loader = data.DataLoader(testData,batch_size=batch_size)

In [11]:
class fruitModel(nn.Module):
    def __init__(self, num_classes=155):
        super(fruitModel, self).__init__()
        self.features=nn.Sequential(nn.Conv2d(3,64,kernel_size=11,stride=2,padding=2),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=3,stride=2),
                                    
                                   nn.Conv2d(64,192,kernel_size=5,padding=2),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=3,stride=2),
                                    
                                   nn.Conv2d(192,384,kernel_size=3,padding=1),
                                   nn.ReLU(),
                                   nn.Conv2d(384,256,kernel_size=3,padding=1),
                                   nn.ReLU(),
                                   nn.Conv2d(256,256,kernel_size=3,padding=1),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=3, stride=2),
                                   )
        self.avgpool = nn.AdaptiveAvgPool2d((6,6))
        self.classifier = nn.Sequential(nn.Dropout(),
                                       nn.Linear(256*6*6,4096),
                                       nn.ReLU(),
                                       nn.Dropout(),
                                       nn.Linear(4096,4096),
                                       nn.ReLU(),
                                       nn.Linear(4096, num_classes))
    def forward(self,x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = self.classifier(x)
        return x

In [12]:
cnet = fruitModel()
cnet.to(devices)
optimizer = optim.Adam(cnet.parameters(),lr=0.001)

In [14]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20,device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output, targets)
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output),dim=1)[1],targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)
        if epoch == 0:
            print(device)
        print('Epoch: {},Training Loss: {:.2f}, Validation Loss: {:.2f}, accuray = {:.2f}'.format(epoch + 1 , training_loss, valid_loss, num_correct / num_examples))

In [15]:
train(cnet,optimizer,torch.nn.CrossEntropyLoss(),train_data_loader,val_data_loader,device=devices)

ValueError: hue_factor (0.5606355667114258) is not in [-0.5, 0.5].